### Requirements

In [3]:
pip install --upgrade huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.2 MB/s eta 0:00:00


In [6]:
pip install transformers

In [7]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.9 MB/s eta 0:00:00


In [8]:
!pip3 install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.6 MB/s eta 0:00:00a 0:00:01


### Import

In [9]:
import pandas as pd

In [10]:
import torch

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

### Loading Reference Data

In [13]:
df_two = pd.read_csv("KnowledgeTransfers-two_sent.csv")

In [14]:
### Selecting all annotated cases of family relations
df_fam_two = df_two[df_two.relation.isin(["parent_of", "child_of", "siblings", "spouses"])].copy(deep=True)
df_fam_two.reset_index(drop=True, inplace=True)

In [15]:
df_fam_two.relation.value_counts()

parent_of    29
child_of     26
siblings     23
spouses      11
Name: relation, dtype: int64

In [16]:
df_fam_two.shape

(89, 11)

In [17]:
df_fam_two.drop([11, 19, 20, 22], inplace=True)
df_fam_two.reset_index(drop=True, inplace=True)

In [18]:
df_test = df_fam_two[:14].copy(deep=True)

### Loading Model

In [19]:
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  #bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type = "nf4",
  bnb_4bit_compute_dtype = torch.float16
)

In [20]:
model_id_1 = "meta-llama/Llama-2-7b-chat-hf"

tokenizer_1 = AutoTokenizer.from_pretrained(model_id_1, use_auth_token=True, use_fast=True)
model_7b = AutoModelForCausalLM.from_pretrained(model_id_1, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

hf_7b = pipeline("text-generation", model=model_7b, tokenizer=tokenizer_1, temperature = 0.001)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [21]:
model_id_2 = "meta-llama/Llama-2-13b-chat-hf"

tokenizer_2 = AutoTokenizer.from_pretrained(model_id_2, use_auth_token=True, use_fast=True)
model_13b = AutoModelForCausalLM.from_pretrained(model_id_2, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

hf_13b = pipeline("text-generation", model=model_13b, tokenizer=tokenizer_2, temperature = 0.001)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Creating Prompt Templates

In [82]:
def generate(prompt, max_length=1024, pipe=hf_13b, **kwargs):
    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

In [83]:
def prompt_with_examples(prompt, examples=[]):

    # Start with the initial part of the prompt
    full_prompt = "<s>[INST]\n"

    # Add each example to the prompt
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

#### Building Examples for Few-Shot Prompts

In [84]:
example_parent = "LUCIE einen Mantelsack tragend, zu Karl. Laß Er's nur, es ist nicht schwer; aber nehm Er meiner Mutter die Schachtel ab. POSTMEISTERIN. Ihre Dienerin, meine Frauenzimmer!"

In [85]:
example_child = "Lucie tritt auf. FERNANDO. O meine Tochter! LUCIE. Lieber, bester Vater!"

In [86]:
example_sibling = "Deine Frau? Hoffmann. Nein, ihre Schwester. Hörtest Du nicht, wie sie mich betitelte? Loth."

In [87]:
example_spouses = "Es wird so Nacht! FERNANDO. Und dieses Weib ist meine Frau! – STELLA"

In [88]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_parent}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_parent = "parent_of"
example_parent = (example_prompt, example_response_parent)

In [89]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_child}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_child = "child_of"
example_child = (example_prompt, example_response_child)

In [90]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_sibling}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_sibling = "siblings"
example_sibling = (example_prompt, example_response_sibling)

In [91]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_spouses}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_spouses = "spouses"
example_spouses = (example_prompt, example_response_spouses)

In [92]:
examples = [
    example_parent,
    example_child,
    example_sibling,
    example_spouses
    ]

### Runing Prompts on Two-Sentences

In [93]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [94]:
###Zero Shot
results = []
for text in df_fam_two.context:
  drama_snippet = text

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", "child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [95]:
results_1 = []
for text in results:
  if "parent_of" in text:
    results_1.append("parent_of")
  elif "child_of" in text:
    results_1.append("child_of")
  elif "siblings" in text:
    results_1.append("siblings")
  elif "spouses" in text:
    results_1.append("spouses")
  elif "Parent_of" in text:
    results_1.append("parent_of")
  elif "Child_of" in text:
    results_1.append("child_of")
  elif "Siblings" in text:
    results_1.append("siblings")
  elif "Spouses" in text:
    results_1.append("spouses")
  else:
    results_1.append("NaN")

In [96]:
df_fam_two_zero = df_fam_two.copy(deep=True)

In [97]:
len(results_1)

85

In [98]:
results_se = pd.Series(results_1)

In [99]:
df_fam_two_zero.famrel_pred = results_se

<ipython-input-99-d01aaceac70e>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_two_zero.famrel_pred = results_se


In [100]:
ground_truth = df_fam_two_zero.relation
predictions = df_fam_two_zero.famrel_pred

In [101]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions)])),
                      columns = ["F1", "Precision", "Recall", "Accuracy"])

,F1,Precision,Recall,Accuracy
0,0.444563,0.509458,0.470588,0.470588


In [102]:
###Zero Shot 2
results_0 = []
for text in df_fam_two.context:
  drama_snippet = text

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of the following labels:
  A: "child_of"
  B: "parent_of"
  C: "siblings"
  D: "spouses".
  JUST name the label, do NOT generate any more text!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results_0.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [103]:
results_01 = []
for text in results_0:
  if "parent_of" in text:
    results_01.append("parent_of")
  elif "child_of" in text:
    results_01.append("child_of")
  elif "siblings" in text:
    results_01.append("siblings")
  elif "spouses" in text:
    results_01.append("spouses")
  elif "Parent_of" in text:
    results_01.append("parent_of")
  elif "Child_of" in text:
    results_01.append("child_of")
  elif "Siblings" in text:
    results_01.append("siblings")
  elif "Spouses" in text:
    results_01.append("spouses")
  else:
    results_01.append("NaN")

In [104]:
df_fam_two_zero_v2 = df_fam_two.copy(deep=True)

In [105]:
len(results_01)

85

In [106]:
results_se_v2 = pd.Series(results_01)

In [107]:
df_fam_two_zero_v2.famrel_pred = results_se_v2

<ipython-input-107-f871e1248415>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_two_zero_v2.famrel_pred = results_se_v2


In [108]:
ground_truth_v2 = df_fam_two_zero_v2.relation
predictions_v2 = df_fam_two_zero_v2.famrel_pred

In [109]:
pd.DataFrame(list(zip([f1_score(ground_truth_v2, predictions_v2, average="weighted")],
                      [precision_score(ground_truth_v2, predictions_v2, average="weighted")],
                      [recall_score(ground_truth_v2, predictions_v2,  average="weighted")],
                      [accuracy_score(ground_truth_v2, predictions_v2)])),
                      columns = ["F1", "Precision", "Recall", "Accuracy"])

,F1,Precision,Recall,Accuracy
0,0.51043,0.506619,0.529412,0.529412


In [110]:
###Few-Shot
results_2 = []
for text in df_fam_two.context:
  drama_snippet = text

  prompt = f"""
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  """

  prompt_exam = prompt_with_examples(prompt, examples)

  result = generate(prompt_exam)
  results_2.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [111]:
df_fam_two_few = df_fam_two.copy(deep=True)

In [112]:
results_22 = []
for text in results_2:
  if "parent_of" in text:
    results_22.append("parent_of")
  elif "child_of" in text:
    results_22.append("child_of")
  elif "siblings" in text:
    results_22.append("siblings")
  elif "spouses" in text:
    results_22.append("spouses")
  elif "Parent_of" in text:
    results_22.append("parent_of")
  elif "Child_of" in text:
    results_22.append("child_of")
  elif "Siblings" in text:
    results_22.append("siblings")
  elif "Spouses" in text:
    results_22.append("spouses")
  else:
    results_22.append("NaN")

In [113]:
results_2_se = pd.Series(results_22)

In [114]:
df_fam_two_few.famrel_pred = results_2_se

<ipython-input-114-884115a20383>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_two_few.famrel_pred = results_2_se


In [115]:
df_fam_two_few.famrel_pred = df_fam_two_few.famrel_pred.replace('mother', 'parent_of')
df_fam_two_few.famrel_pred = df_fam_two_few.famrel_pred.replace('daughter', 'child_of')

In [116]:
ground_truth_few = df_fam_two_few.relation
predictions_few = df_fam_two_few.famrel_pred

In [117]:
pd.DataFrame(list(zip([f1_score(ground_truth_few, predictions_few, average="weighted")],
                      [precision_score(ground_truth_few, predictions_few, average="weighted")],
                      [recall_score(ground_truth_few, predictions_few,  average="weighted")],
                      [accuracy_score(ground_truth_few, predictions_few,)])),
                      columns = ["F1", "Precision", "Recall", "Accuracy"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Precision,Recall,Accuracy
0,0.378128,0.362597,0.4,0.4


In [118]:
###Zero Shot with Names
results_3 = []
for index, row in df_fam_two.iterrows():
  person_1 = row["arg1"]
  person_2 = row["arg2"]
  drama_snippet = row["context"]

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship between {person_1} and {person_2} is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results_3.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [119]:
results_4 = []
for text in results_3:
  if "parent_of" in text:
    results_4.append("parent_of")
  elif "child_of" in text:
    results_4.append("child_of")
  elif "siblings" in text:
    results_4.append("siblings")
  elif "spouses" in text:
    results_4.append("spouses")
  elif "Parent_of" in text:
    results_4.append("parent_of")
  elif "Child_of" in text:
    results_4.append("child_of")
  elif "Siblings" in text:
    results_4.append("siblings")
  elif "Spouses" in text:
    results_4.append("spouses")
  else:
    results_4.append("NaN")

In [120]:
df_fam_two_pers_zero = df_fam_two.copy(deep=True)

In [121]:
results_se_3 = pd.Series(results_4)

In [122]:
df_fam_two_pers_zero.famrel_pred = results_se_3

<ipython-input-122-ed4ba08e3ec1>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_two_pers_zero.famrel_pred = results_se_3


In [123]:
ground_truth_3 = df_fam_two_pers_zero.relation
predictions_3 = df_fam_two_pers_zero.famrel_pred

In [124]:
pd.DataFrame(list(zip([f1_score(ground_truth_3, predictions_3, average="weighted")],
                       [precision_score(ground_truth_3, predictions_3, average="weighted")],
                      [recall_score(ground_truth_3, predictions_3,  average="weighted")],
                      [accuracy_score(ground_truth_3, predictions_3)])),
                      columns = ["F1", "Precision", "Recall", "Accuracy"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Precision,Recall,Accuracy
0,0.665363,0.700392,0.647059,0.647059


In [65]:
###Few Shot with Names

In [125]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Madame Sommer und Lucie in the following German {example_parent}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_parent = "parent_of"
example_parent_pers = (example_prompt_pers, example_response_parent)

In [126]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Lucie and Fernando in the following German {example_child}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_child = "child_of"
example_child_pers = (example_prompt_pers, example_response_child)

In [127]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Fernando and Madame Sommer in the following German {example_sibling}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_sibling = "siblings"
example_sibling_pers = (example_prompt_pers, example_response_sibling)

In [128]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Helene and the NaN in the following German {example_spouses}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_spouses = "spouses"
example_spouses_pers = (example_prompt_pers, example_response_spouses)

In [129]:
examples_pers = [
    example_parent_pers,
    example_child_pers,
    example_sibling_pers,
    example_spouses_pers
    ]

In [130]:
###Few-Shot with Names
results_5 = []
for index, row in df_fam_two.iterrows():
  person_1 = row["arg1"]
  person_2 = row["arg2"]
  drama_snippet = row["context"]

  prompt_pers = f"""
  <s>[INST]\n
  What kind of family relationship between {person_1} and {person_2} is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  prompt_exam_pers = prompt_with_examples(prompt_pers, examples_pers)

  result = generate(prompt_exam_pers)
  results_5.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1030, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1204, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1048, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warning

In [131]:
df_fam_two_pers_few = df_fam_two.copy(deep=True)

In [132]:
results_4_se = pd.Series(results_5)

In [133]:
df_fam_two_pers_few.famrel_pred = results_4_se

<ipython-input-133-b5c3dbd79d77>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_two_pers_few.famrel_pred = results_4_se


In [134]:
ground_truth_4 = df_fam_two_pers_few.relation
predictions_4 = df_fam_two_pers_few.famrel_pred

In [135]:
pd.DataFrame(list(zip([f1_score(ground_truth_4, predictions_4, average="weighted")],
                      [precision_score(ground_truth_4, predictions_4, average="weighted")],
                      [recall_score(ground_truth_4, predictions_4,  average="weighted")],
                      [accuracy_score(ground_truth_4, predictions_4)])),
                      columns = ["F1", "Precision", "Recall", "Accuracy"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Precision,Recall,Accuracy
0,0.682046,0.744432,0.658824,0.658824
